In [37]:
import pandas as pd
import fnmatch
import numpy as np

In [38]:
# Import the cleaned crime data
crime_df = pd.read_csv("crime_data_cleaned.csv").drop("Unnamed: 0", axis=1)
crime_df

,Region,Antal,/100 000 inv,Date
0,Gotlands län,..,..,1998-12
1,Gotlands län,..,..,2002-10
2,Gotlands län,..,..,2007-11
3,Gotlands län,..,..,2011-11
4,Gotlands län,..,..,2012-02
...,...,...,...,...
2515,Östergötlands län,40,9,2008-03
2516,Östergötlands län,38,9,2010-01
2517,Östergötlands län,38,9,2010-03
2518,Östergötlands län,38,9,2012-12


In [39]:
# Import the weather data with county information.
weather_regions_df = pd.read_csv("weather_county_data.csv").drop("Unnamed: 0", axis=1)
weather_regions_df

,location,date,snow_depth,Region
0,Abisko,1995-01-01,17,Norrbottens län
1,Abisko,1995-01-02,17,Norrbottens län
2,Abisko,1995-01-03,17,Norrbottens län
3,Abisko,1995-01-04,17,Norrbottens län
4,Abisko,1995-01-05,17,Norrbottens län
...,...,...,...,...
2044544,Övre Soppero,2021-08-28,0,Norrbottens län
2044545,Övre Soppero,2021-08-29,0,Norrbottens län
2044546,Övre Soppero,2021-08-30,0,Norrbottens län
2044547,Övre Soppero,2021-08-31,0,Norrbottens län


In [40]:
# Update names of columns on weather data & remove days from date
weather_regions_df.columns = ["Location", "Date", "Snow_depth", "Region"]
weather_regions_df["Date"] = weather_regions_df["Date"].apply(lambda x: x[:-3])
weather_regions_df

,Location,Date,Snow_depth,Region
0,Abisko,1995-01,17,Norrbottens län
1,Abisko,1995-01,17,Norrbottens län
2,Abisko,1995-01,17,Norrbottens län
3,Abisko,1995-01,17,Norrbottens län
4,Abisko,1995-01,17,Norrbottens län
...,...,...,...,...
2044544,Övre Soppero,2021-08,0,Norrbottens län
2044545,Övre Soppero,2021-08,0,Norrbottens län
2044546,Övre Soppero,2021-08,0,Norrbottens län
2044547,Övre Soppero,2021-08,0,Norrbottens län


In [41]:
# The weather data is coded with a number depending on how much snow is on the
# ground. We are only interested in days when there is snow so we can
# disregard the other days. Days with snow is coded 1, no snow is coded 0.

def snow_day(x):
    if (x == 17) | (x == 14)|(x == 17) | (x == 18) | (x == 19):
        return 1
    else:
        return 0

In [42]:
# We can now drop unneccecary columns.
weather_regions_df = weather_regions_df.drop("Snow_depth", axis=1)

In [43]:
# Group and sum the number of snow days.
weather_regions_snow_days = weather_regions_df.groupby(["Location", "Date", "Region"]).sum()

In [44]:
weather_regions_snow_days.reset_index(inplace=True)
weather_regions_snow_days

,Location,Date,Region
0,Abisko,1995-01,Norrbottens län
1,Abisko,1995-02,Norrbottens län
2,Abisko,1995-03,Norrbottens län
3,Abisko,1995-04,Norrbottens län
4,Abisko,1995-05,Norrbottens län
...,...,...,...
85546,Övre Soppero,2021-05,Norrbottens län
85547,Övre Soppero,2021-06,Norrbottens län
85548,Övre Soppero,2021-07,Norrbottens län
85549,Övre Soppero,2021-08,Norrbottens län


In [45]:
# Create the final merged dataset by merging the crime and
# snow dataframes.
merged_df = crime_df.merge(weather_regions_snow_days, on=["Region", "Date"])
merged_df

,Region,Antal,/100 000 inv,Date,Location
0,Gotlands län,..,..,1998-12,Forse
1,Gotlands län,..,..,1998-12,Hejnum
2,Gotlands län,..,..,1998-12,Hemse
3,Gotlands län,..,..,1998-12,Hoburg D
4,Gotlands län,..,..,1998-12,Sanda
...,...,...,...,...,...
33622,Östergötlands län,41,9,2013-03,Tjällmo D
33623,Östergötlands län,41,9,2013-03,Vadstena D
33624,Östergötlands län,41,9,2013-03,Åkroken D
33625,Östergötlands län,41,9,2013-03,Ödeshög D


In [54]:
# Now when the data has been merged we now need to aggregate the number of
# snow days and crimes each month. The following script goes through the 
# data and creates the artifical number 'snow_count'. Snow count
# is the number of snow observations divided by the number of stations.
# The script also aggregates the number of crimes.

prev_date = ""
prev_region = ""
snow_count = 0
crime_count = ""
stations_count = 0
max_index = merged_df.shape[0]-1

data = []

for i, row in merged_df.iterrows():
    
    curr_region = row['Region']
    curr_date = row['Date']
    crime = row['Antal']
    snow = row['Snow_days']
    
    if(i == 0):
        prev_date = curr_date
        prev_region = curr_region
        stations_count = 1
    
    if (prev_date != curr_date or prev_region != curr_region):
        
        converted_date = pd.to_datetime(prev_date, format='%Y-%m', errors='ignore')
        to_append = [prev_region, converted_date, crime_count, (snow_count / stations_count)]
        data.append(to_append)
        
        prev_date = curr_date
        prev_region = curr_region
        
        snow_count = 0
        stations_count = 1
        crime_count = crime
        snow_count = snow
    elif (i == max_index):
        to_append = [prev_region, prev_date, crime_count, (snow_count / stations_count)]
        data.append(to_append)
    else:
        crime_count = crime
        snow_count = snow_count + snow
        stations_count = stations_count + 1      
        
aggregated_df = pd.DataFrame(data, columns=['region', 'date', 'crime_count', 'snow_count'])

/var/folders/cp/4l5cyd252gl8gyd90gw38swr0000gn/T/ipykernel_56180/892840870.py:49: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype={value.dtype})
  aggregated_df = pd.DataFrame(data, columns=['region', 'date', 'crime_count', 'snow_count'])


In [55]:
# Create a new index for crime count.
index_names = aggregated_df[ aggregated_df['crime_count'] == ".." ].index
  
# Drop row indexes from dataFrame
aggregated_df.drop(index_names, inplace = True)

# Convert numbers to integers
aggregated_df["crime_count"] = aggregated_df["crime_count"].apply(lambda x: float(x))
aggregated_df

,region,date,crime_count,snow_count
9,Jämtlands län,1995-01-01,1.0,11.800000
10,Jämtlands län,1995-03-01,2.0,11.480000
11,Jämtlands län,1995-11-01,2.0,4.842105
12,Jämtlands län,1996-12-01,2.0,13.800000
13,Norrbottens län,1995-12-01,4.0,9.818182
...,...,...,...,...
2469,Östergötlands län,2008-03-01,40.0,9.400000
2470,Östergötlands län,2010-01-01,38.0,29.500000
2471,Östergötlands län,2010-03-01,38.0,24.941176
2472,Östergötlands län,2012-12-01,38.0,24.800000


In [56]:
aggregated_df.to_csv("final_dataset.csv")